# Summary information about Process Leaf Nodes

In [29]:
# Import packages used in notebooks
import altair as alt
import duckdb
import wintapgraph as wg
import networkx as nx
%load_ext magic_duckdb

alt.data_transformers.disable_max_rows()

The magic_duckdb extension is already loaded. To reload it, use:
  %reload_ext magic_duckdb


DataTransformerRegistry.enable('default')

In [3]:
# Initialize an in-memory db. Save reference in a variable and then set magic-duckdb environment. Result is ability to use the same DB instance from python code and %dql/%%dql magics.
# Also create views for every top-level type found in the current dataset.
con = duckdb.connect()
%dql -co con
# Display the list of tables/views
%dql show tables
# Only uses process table
%dql create view process_leaf_nodes as from '/data/process_leaf_nodes.parquet'

,Count


In [5]:
%dql summarize process_leaf_nodes

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,time_bucket_start,TIMESTAMP,2023-11-05 00:00:00,2023-11-19 23:55:00,4359,None,None,None,None,None,3863136,0.00
1,time_bucket_end,TIMESTAMP,2023-11-05 00:05:00,2023-11-20 00:00:00,4359,None,None,None,None,None,3863136,0.00
2,hostname,VARCHAR,ACME-DC1,ACME-WS-UVF,26,None,None,None,None,None,3863136,0.00
3,pid_hash,VARCHAR,00000AD4E601CD6811A249DE5566D701,FFFFFABC708750507DC23EF8C8BBA50F,2632148,None,None,None,None,None,3863136,0.00
4,process_name,VARCHAR,/powershell.exe,xmlcatalog.exe,331,None,None,None,None,None,3863136,2.20
5,process_started,TIMESTAMP WITH TIME ZONE,2023-11-01 12:10:11.5-04,2023-11-19 23:59:43.683744-05,2501797,None,None,None,None,None,3863136,2.20
6,process_term,TIMESTAMP WITH TIME ZONE,2023-11-05 00:00:00.474481-04,2023-11-20 17:16:35.707756-05,1812661,None,None,None,None,None,3863136,19.75
7,active,INTEGER,0,1,2,0.7804444368513043,0.4139456029414491,1,1,1,3863136,0.00
8,started,INTEGER,0,1,2,0.6503074186360511,0.4768728748514264,0,1,1,3863136,0.00
9,term,INTEGER,0,1,2,0.4748243396038866,0.49936584852574245,0,0,1,3863136,0.00


In [6]:
%dql select started,active,term,count(*) from process_leaf_nodes group by all

,started,active,term,count_star()
0,0,0,1,85094
1,0,1,0,1240671
2,0,1,1,25145
3,1,0,0,763079
4,1,1,0,25075
5,1,1,1,1724072


In [12]:
%%dql -o plf_df
SELECT
	time_bucket_start,
	count(distinct hostname) num_hosts,
	count(distinct process_name) num_process_names,
	count(*) filter (started = 1) num_started,
	count(*) filter (active = 1) num_active,
	count(*) filter (term = 1) num_term,
from
	process_leaf_nodes
group by
	time_bucket_start

,time_bucket_start,num_hosts,num_process_names,num_started,num_active,num_term
0,2023-11-05 00:10:00,26,35,715,621,504
1,2023-11-05 01:05:00,26,21,641,568,512
2,2023-11-05 01:35:00,26,23,599,546,432
3,2023-11-05 02:15:00,26,23,600,555,467
4,2023-11-05 03:00:00,26,23,670,617,489
...,...,...,...,...,...,...
4315,2023-11-17 00:30:00,25,24,625,770,436
4316,2023-11-17 00:35:00,25,24,568,730,425
4317,2023-11-17 01:25:00,25,24,641,759,457
4318,2023-11-17 01:40:00,25,24,617,759,460


In [25]:
plf_df_melt=plf_df.melt('time_bucket_start', var_name='proc_event', value_name='count')

In [31]:

alt.Chart(plf_df_melt).mark_line().encode(
    x='time_bucket_start',
    y=alt.Y('count'), #,scale=alt.Scale(type="symlog")),
    color='proc_event',
    tooltip=['time_bucket_start:T']
).properties(
    title='Process starts for leaf nodes',
    width=1200,
    height=400
).interactive()

alt.Chart(...)